In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import pandas as pd
from datetime import datetime

def experiment_version_frequency(experiment_name, start_time, end_time, tracking_uri="http://localhost/mlflow-service"):
    """
    Prints experiment id, run count, and average frequency of model versions for a given experiment name and time range.
    start_time and end_time should be in 'YYYY-MM-DD HH:MM:SS' format.
    Average frequency is now calculated per second.
    """
    client = MlflowClient(tracking_uri=tracking_uri)
    exp = client.get_experiment_by_name(experiment_name)
    if exp is None:
        print(f"Experiment '{experiment_name}' not found.")
        return
    runs = client.search_runs([exp.experiment_id])
    # Convert start/end time to timestamps
    start_ts = int(pd.Timestamp(start_time).timestamp() * 1000)
    end_ts = int(pd.Timestamp(end_time).timestamp() * 1000)
    filtered_runs = [run for run in runs if start_ts <= run.info.start_time <= end_ts]
    print(f"Experiment ID: {exp.experiment_id}")
    print(f"Run count: {len(filtered_runs)}")
    if filtered_runs:
        times = [run.info.start_time for run in filtered_runs]
        times.sort()
        # Calculate average frequency (runs per second)
        duration = (max(times) - min(times)) / 1000  # seconds
        avg_freq = len(filtered_runs) / duration if duration > 0 else len(filtered_runs)
        print(f"Average frequency: {avg_freq:.6f} runs per second")
        print("Run IDs and start times:")
        for run in filtered_runs:
            dt = datetime.fromtimestamp(run.info.start_time / 1000)
            print(f"Run ID: {run.info.run_id}, Start Time: {dt}")
    else:
        print("No runs found in the specified time range.")

# Example usage:
experiment_version_frequency("AGR_a-HT-old-2000", "2025-07-19 00:00:00", "2025-07-19 23:59:59")
